### 외부 라이브러리를 사용하는 컴포넌트 만들기
- ubeflow에서 컴포넌트들은 각각 컨테이너 위에서 서로 독립적으로 실행. 즉, 같은 메모리를 공유하고 있지 않기 때문에, 보통의 파이썬 함수에서 사용하는 방식과 같이 객체를 전달할 수 없음.

- 컴포넌트 간에 넘겨 줄 수 있는 정보는 json으로만 가능

- Kubeflow에서는 이를 해결하기 위해 json-serializable 하지 않은 타입의 객체는 메모리 대신 파일에 데이터를 저장한 뒤, 그 파일을 이용해 정보를 전달

- 이를 해결하기 위한 방법은, 데이터를 생성하고 반환하는 컴포넌트에서는 data_path: OutputPath()를, 데이터를 받는 컴포넌트에서는 data_path: InputPath()을 argument로 생성

- 이렇게 만든 후 파이프라인에서 서로 연결을 하면 kubeflow에서 필요한 경로를 자동으로 생성

In [ ]:
from kfp.components import InputPath, OutputPath, create_component_from_func


@create_component_from_func
def train_from_csv(
    train_data_path: InputPath("csv"),
    train_target_path: InputPath("csv"),
    model_path: OutputPath("dill"),
    kernel: str,
):
    import dill
    import pandas as pd

    from sklearn.svm import SVC

    train_data = pd.read_csv(train_data_path)
    train_target = pd.read_csv(train_target_path)

    clf = SVC(kernel=kernel)
    clf.fit(train_data, train_target)

    with open(model_path, mode="wb") as file_writer:
        dill.dump(clf, file_writer)

### 데이터를 생성하는 Component 제작

In [ ]:
@create_component_from_func
def load_iris_data(
    data_path: OutputPath("csv"),
    target_path: OutputPath("csv"),
):
    import pandas as pd
    from sklearn.datasets import load_iris

    iris = load_iris()

    data = pd.DataFrame(iris["data"], columns=iris["feature_names"])
    target = pd.DataFrame(iris["target"], columns=["target"])

    data.to_csv(data_path, index=False)
    target.to_csv(target_path, index=False)

### 파이프라인 만들기
* **주의!** 입력과 출력에서 받는 argument중 경로와 관련된 것들에 _path 접미사가 모두 사라짐. iris_data.outputs["data_path"] 가 아닌 iris_data.outputs["data"] 으로 접근하는 것을 확인할 수 있음.

* 이는 kubeflow에서 정한 법칙으로 InputPath 와 OutputPath 으로 생성된 경로들은 파이프라인에서 접근할 때는 _path 접미사를 생략하여 접근하기 때문

In [ ]:
from kfp.dsl import pipeline


@pipeline(name="complex_pipeline")
def complex_pipeline(kernel: str):
    iris_data = load_iris_data()
    model = train_from_csv(
        train_data=iris_data.outputs["data"],
        train_target=iris_data.outputs["target"],
        kernel=kernel,
    )

### 외부 라이브러리를 포함한 컴포넌트 다루기
* 방금 제작한 파이프라인을 실행하면 오류가 발생함. 이는 프로그램이 python만 설치된 이미지에서 실행되기 때문임.

* create_component_from_func의 `base_image` argument 또는 `packages_to_install`를 사용해 이 문제를 해결 할 수 있음.

* base_image는 dockerfile을 일일이 만들어야하므로 `packages_to_install`를 사용함.

In [1]:
from functools import partial
from kfp.components import InputPath, OutputPath, create_component_from_func

@partial(
    create_component_from_func,
    packages_to_install=["dill==0.3.4", "pandas==1.3.4", "scikit-learn==1.0.1"],
)
def train_from_csv(
    train_data_path: InputPath("csv"),
    train_target_path: InputPath("csv"),
    model_path: OutputPath("dill"),
    kernel: str,
):
    import dill
    import pandas as pd

    from sklearn.svm import SVC

    train_data = pd.read_csv(train_data_path)
    train_target = pd.read_csv(train_target_path)

    clf = SVC(kernel=kernel)
    clf.fit(train_data, train_target)

    with open(model_path, mode="wb") as file_writer:
        dill.dump(clf, file_writer)

if __name__ == "__main__":
    train_from_csv.component_spec.save("train_from_csv.yaml")